# The Laplace approximation

### Load packages

In [4]:
from numpy.polynomial.hermite import hermgauss
from matplotlib import pyplot as plt
from numpy.linalg import inv
import pandas as pd
import numpy as np
import scipy

### Randomized simulation data for two sites
- Set random seed
- Generate 10 true betas ranged in (-10, 10)
- Generate 2 sigmas for noise variance in different sites
- Generate data X1 and X2 with size (1000, 10)
- Generate result y1 and y2 with bernoulli distibution 

In [5]:
np.random.seed(1)
true_beta = (np.random.rand(10,1) - np.random.rand(10,1)) * 10
true_sigma = np.random.rand(2)
X1 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p1 = 1 / (1 + np.exp(-(X1 @ true_beta + np.random.normal(0, true_sigma[0], 1000).reshape(1000, 1))))
y1 = np.random.binomial(1,p1)
X2 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p2 = 1 / (1 + np.exp(-(X2 @ true_beta + np.random.normal(0, true_sigma[1], 1000).reshape(1000, 1))))
y2 = np.random.binomial(1,p2)

### Definitions

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta_0}+\mu_{i0})}{1 + \exp{(X_{ij}^\top\beta_0}+\mu_{i0})}$$

In [6]:
def Pi(x, beta_0, mu):
    return np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))

Notation for $g(\mu_{i0})$:

$$g(\mu_{i0};\beta_0)=\sum_{j=1}^{n_i}\left[y_{ij}\log\pi_{ij}+(1-y_{ij})\log(1-\pi_{ij})\right]+\log\phi(\mu_{i0};\theta_0)$$

In [7]:
def g(x, y, mu, beta_0, tau=1):
    g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \
    + np.log((np.sqrt(2 * np.pi) * tau)**(-1) * np.exp(-mu**2/(2 * tau**2)))
    return g

Notation for $g_\beta(\mu_{i0})$:

$$\dfrac{\partial g}{\partial \beta_0}=\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})$$

In [19]:
def g_b(x, y, mu, beta_0, tau = 1):
    return np.sum(x * y - x * Pi(x, beta_0, mu), axis = 0)

Notation for $g_\mu(\mu_{i0})$:

$$\dfrac{\partial g}{\partial \mu_{i0}}=\sum_{j=1}^{n_i}(y_{ij}-\pi_{ij})-\dfrac{\mu_{i0}}{\tau_0^2}$$

In [8]:
def g_u(x, y, mu, beta_0, tau = 1):
    return sum(y - Pi(x, beta_0, mu)) - mu/tau**2

Notation for $g_{\mu\mu}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i0}^2}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\mu_{i0}}-\dfrac{1}{\tau_0^2}$$

In [14]:
def g_uu(x, y, mu, beta_0, tau = 1):
    return sum(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2))) - 1/tau**2

Notation for $g_{\mu\beta}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i0}\partial \beta_0}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\beta_0}$$

In [95]:
def g_ub(x, y, mu, beta_0, tau = 1):
    return np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2)), axis = 0)

Notation for $g_{\beta\beta}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \beta_0^2}=-\sum_{j=1}^{n_i}X_{ij}\dfrac{\partial\pi_{ij}}{\partial\beta_0}$$

In [100]:
def g_bb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
        * (np.exp(x[i] @ beta_0 + mu) / (1 + np.exp(x[i] @ beta_0 + mu))**2))
    return result

Notation for $g_{\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$\dfrac{\partial g_u(\hat\mu(\beta_0);\beta_0)}{\partial \beta_0}=\hat\mu_\beta(\beta_0)g_{\mu\mu}(\hat\mu_{i0})+g_{\mu\beta}(\hat\mu_{i0})=0$$

Notation for $g_{\mu\mu\mu}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i0}^2}$$

In [16]:
def g_uuu(x, y, mu, beta_0, tau = 1):
    return sum(- np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)))

Notation for $g_{\mu\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i0}\partial\beta_0}$$

In [17]:
def g_uub(x, y, mu, beta_0, tau = 1):
    return np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)), axis = 0)

Notation for $g_{\mu\beta\beta}(\hat\mu_{i0};\beta_0)$:

$$\dfrac{\partial^3 g}{\partial \mu_{i0}\partial \beta_0^2}=-\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\beta_0^2}$$

In [84]:
def g_ubb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                         * (np.exp(x[i] @ beta_0 + mu) * (np.exp(x[i] @ beta_0 + mu) - 1)\
                                       / (1 + np.exp(x[i] @ beta_0 + mu))**3))
    return result

Notation for $g_{\mu\mu\mu\mu}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu\mu}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}^3}$$

In [70]:
def g_uuuu(x, y, mu, beta_0, tau = 1):
    result = sum(- (np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)\
                )
    return result

Notation for $g_{\mu\mu\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}^2\partial\beta_0}$$

In [71]:
def g_uuub(x, y, mu, beta_0, tau = 1):
    result = np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)), axis = 0)
    return result

Notation for $g_{\mu\mu\beta\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\beta\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}\partial\beta^2_0}$$

In [72]:
def g_uubb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                             * ((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)))
        
    return result

Notation for $\hat\omega$:

$$\hat\omega=\sqrt{-\dfrac{1}{g_{\mu\mu}(\hat\mu_{i0})}}$$

In [10]:
def omega(x, y, mu, beta_0, tau = 1):
    return np.sqrt(-1/g_uu(x, y, mu, beta_0))

Notation for $\hat\omega_\beta$:

$$\dfrac{\partial\hat\omega}{\partial\beta}=\dfrac{1}{2}\hat\omega^3(g_{\mu\mu\mu}\hat\mu_\beta + g_{\mu\mu\beta})$$

In [18]:
def omega_b(x, y, mu, beta_0, tau = 1):
    return 0.5 * omega(x, y, mu, beta_0, tau)**3 * (g_uuu(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau)\
                                                    + g_uub(x, y, mu, beta_0, tau))

Notation for $\hat\mu_\beta$

$$\hat\mu_\beta(\beta_0)=-\dfrac{g_{\mu\beta}(\hat\mu_{i0};\beta_0)}{g_{\mu\mu}(\hat\mu_{i0};\beta_0)}=\hat\omega^2(\beta_0)g_{\mu\beta}(\hat\mu_{i0}(\beta_0);\beta_0)$$

In [15]:
def mu_b(x, y, mu, beta_0, tau = 1):
    return omega(x, y, mu, beta_0, tau)**2 * g_ub(x, y, mu, beta_0, tau)

Notation for $\hat\mu_{\beta\beta'}$:

$$\hat\mu_{\beta\beta'}(\beta_0)=\hat\omega^2(\hat\mu_\beta\hat\mu_{\beta'} g_{\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\beta'}+\hat\mu_{\beta'}g_{\mu\mu\beta}+g_{\mu\beta\beta'})$$

In [78]:
def mu_bb(x, y, mu, beta_0, tau = 1):
    result = omega(x, y, mu, beta_0, tau)**2 * (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                              @ mu_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
                                              * g_uuu(x, y, mu, beta_0, tau)\
                                              + 2 * mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                              @ g_uub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
                                              + g_ubb(x, y, mu, beta_0, tau))
    return result

Notation for $\hat\omega_{\beta\beta'}$:

$$\dfrac{\partial^2}{\partial\beta\partial\beta'}\hat\omega(\beta_0)=\frac{3}{4}\hat\omega^5(\hat\mu_{\beta'}g_{\mu\mu\mu}+g_{\mu\mu\beta'})(\hat\mu_{\beta}g_{\mu\mu\mu}+g_{\mu\mu\beta})+\dfrac{1}{2}\hat\omega^3(\hat\mu_{\beta\beta'}g_{\mu\mu\mu}+\hat\mu_\beta\hat\mu_{\beta'}g_{\mu\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\mu\beta'}+\hat\mu_{\beta\beta'}g_{\mu\mu\mu\beta}+g_{\mu\mu\beta\beta'})$$

In [92]:
def omega_bb(x, y, mu, beta_0, tau = 1):
    result = 3/4 * omega(x, y, mu, beta_0, tau)**5 * (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
                                                      + g_uub(x, y, mu, beta_0, tau)).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
        + g_uub(x, y, mu, beta_0, tau)).reshape(1,x.shape[1]) + 1/2 * omega(x, y, mu, beta_0, tau)**3\
     * (mu_bb(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
        + (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
           @ mu_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]) * g_uuuu(x, y, mu, beta_0, tau))\
        + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1) @ g_uuub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
        + mu_bb(x, y, mu, beta_0, tau)
       )
    return result

Notation for $\mathcal l_i'$:

$$\mathcal l_i' = \dfrac{\partial\mathcal l_i}{\partial\beta_0}=\dfrac{\hat\omega_\beta}{\hat\omega}+g_\mu(\hat\mu_{i0};\beta_0)\hat\mu_\beta(\beta_0)+g_\beta(\hat\mu_{i0};\beta_0)$$

In [22]:
def l_1(x, y, mu, beta_0, tau = 1):
    l1 = omega_b(x, y, mu, beta_0, tau) / omega(x, y, mu, beta_0, tau)\
    + g_u(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau) + g_b(x, y, mu, beta_0, tau)
    return l1

Notation for $l''_i$:

$$l''_i=\dfrac{\partial^2l_i}{\partial\beta_0^2}=\dfrac{\hat\omega_{\beta\beta'}\hat\omega-\hat\omega_\beta\hat\omega_{\beta'}}{\hat\omega^2}+\hat\mu_{\beta\beta'}g_\mu+\hat\mu_\beta(\hat\mu_{\beta'}g_{\mu\mu}+g_{\mu\beta'})+\hat\mu_{\beta'}g_{\mu\beta}+g_{\beta\beta'}$$

In [104]:
def l_2(x, y, mu, beta_0, tau = 1):
    l2 = omega(x, y, mu, beta_0, tau)**(-2) * (omega_bb(x, y, mu, beta_0, tau) * omega(x, y, mu, beta_0, tau)\
                                               - omega_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                               @ omega_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]))\
     + mu_bb(x, y, mu, beta_0, tau) * g_u(x, y, mu, beta_0, tau) + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta_0, tau) * g_uu(x, y, mu, beta_0, tau) + g_ub(x, y, mu, beta_0, tau)).reshape(1,x.shape[1])\
     + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1) @ g_ub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
     + g_bb(x, y, mu, beta_0, tau)
    return l2

Notation for maximizing $\mu$ with respect to $\hat\mu=\arg\max_\mu g(\mu_i0;\beta_0)$

In [149]:
def max_mu(x, y, mu, beta_0, tau=1, max_iter=100):
    for step in range(max_iter):
#         print('Step: ', step, '\n')
        mu_new = mu - g_u(x, y, mu, beta_0, tau)/g_uu(x, y, mu, beta_0, tau)
        diff = mu_new - mu
#         print(diff)
        if np.abs(diff) < 10**(-10):
#             print(mu)
            break;
        mu = mu_new
    return mu

Notation for updating parameters

In [163]:
beta = np.repeat(0, 10).reshape(10,1)
mu = [0.1, 0.1]
tau = 1

print('Initial beta:', beta, "\n")
for step_mu in range(10):
    mu[0] = max_mu(X1, y1, mu[0], beta, tau)
    mu[1] = max_mu(X2, y2, mu[1], beta, tau)
    for step in range(100):
        l1 = l_1(X1, y1, mu[0], beta, tau) + l_1(X2, y2, mu[1], beta, tau)
        l2 = l_2(X1, y1, mu[0], beta, tau) + l_2(X2, y2, mu[1], beta, tau)
        delta = l1 @ inv(l2)
        new_beta = beta - delta.reshape(10,1)
        if max(np.abs(delta)) < 10**(-10):
            break;
        beta = new_beta
#         print('Step ', step+1, ':\n')
#         print('Beta:\n', beta, '\n')
#         print('Diff:\n', delta, '\n')
print('Beta:\n', beta, '\n')

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Beta:
 [[-0.02635409]
 [ 0.2838822 ]
 [-1.98640254]
 [-5.12083263]
 [ 1.09260947]
 [-5.25923557]
 [-2.03314193]
 [-1.86771009]
 [ 2.24603706]
 [ 3.16022686]] 



In [155]:
true_beta

array([[-0.0217251 ],
       [ 0.35104993],
       [-2.04337875],
       [-5.75784864],
       [ 1.19368298],
       [-5.78128915],
       [-2.31044591],
       [-2.13129101],
       [ 2.56380536],
       [ 3.40715245]])

In [161]:
beta

array([[-0.02635409],
       [ 0.2838822 ],
       [-1.98640254],
       [-5.12083263],
       [ 1.09260947],
       [-5.25923557],
       [-2.03314193],
       [-1.86771009],
       [ 2.24603706],
       [ 3.16022686]])